In [1]:
import pandas as pd
from astropy.cosmology import Planck13

import disperse
# import UNet3D
import torch

In [2]:
!ls

139833325838784_galaxies_ascii.txt  _disperse_04  __pycache__	  UNet3D.py
_disperse_01			    disperse.py   README.md
_disperse_02			    MGS_DR7	  SDSS
_disperse_03			    OLD		  tutorial.ipynb


In [3]:
!ls SDSS

DR5_cluster-catalog_v1.1.fits  SDSS_DR16.csv
SDSS_DR12_MGS.dat	       SDSS_DR7.csv
SDSS_DR12_optical_groups.dat   sdss_opt_gr_filtered_RaDecZ.csv


In [4]:
sdss_dr7 = pd.read_csv('SDSS/SDSS_DR7.csv')
sdss_dr7 = sdss_dr7[['ra', 'dec', 'z']]
sdss_dr7.columns = ['RA', 'DEC', 'Z']


coma_ra_int = (120, 280)
coma_dec_int = (-20, 80)
coma_z_int = (0.006, 0.040)
# coma_z_int = (0.006, 0.040)

sdss_dr7 = sdss_dr7[
        (coma_ra_int[0] < sdss_dr7['RA']) & (sdss_dr7['RA'] < coma_ra_int[1]) & \
        (coma_dec_int[0] < sdss_dr7['DEC']) & (sdss_dr7['DEC'] < coma_dec_int[1]) & \
        (coma_z_int[0] < sdss_dr7['Z']) & (sdss_dr7['Z'] < coma_z_int[1])
    ]
sdss_dr7.reset_index(drop=True, inplace=True)

In [5]:
sdss_dr7

,RA,DEC,Z
0,146.71421,-1.041304,0.021314
1,147.17639,-0.354057,0.006436
2,146.85676,-0.274040,0.022017
3,146.61904,-0.434322,0.006087
4,146.75170,-0.408392,0.035831
...,...,...,...
61774,260.39196,31.679209,0.029946
61775,260.64573,31.807949,0.033229
61776,260.73569,31.341986,0.015230
61777,261.10350,31.590881,0.025233


In [6]:
sdss_opt_gr = pd.read_csv('SDSS/sdss_opt_gr_filtered_RaDecZ.csv')
sdss_opt_gr = sdss_opt_gr[sdss_opt_gr['n_gal'] >= 15]
sdss_opt_gr.reset_index(drop=True, inplace=True)

In [7]:
sdss_opt_gr

,n_gal,RA,DEC,Z,r_max,M200,R200
0,180,246.942018,39.401823,0.031119,1.720684,635.8440,1.794430
1,83,247.581739,39.778534,0.028910,1.040201,221.2070,1.262930
2,101,246.935701,40.923333,0.031285,1.363890,307.1370,1.407870
3,85,247.491124,40.610586,0.029515,1.029668,147.2880,1.102600
4,34,246.010700,41.010818,0.030111,1.035922,135.6380,1.072530
...,...,...,...,...,...,...,...
217,22,247.144612,8.529292,0.032348,0.760295,54.8312,0.792467
218,22,179.565240,25.119524,0.015661,0.424296,18.0177,0.549744
219,32,211.719008,55.039101,0.006525,0.544485,19.9125,0.569996
220,15,165.703588,50.720856,0.022642,0.463985,12.5296,0.485982


In [8]:
cosmo = Planck13
H0 = cosmo.H0.value
Om = cosmo.Om0
Ol = 0.69288
Ok = 0.0
print(f'H0 = {cosmo.H0.value}')
print(f'Om = {cosmo.Om0}')
print(f'Ol = {0.69288}')

H0 = 67.77
Om = 0.30712
Ol = 0.69288


In [9]:
DPS = disperse.Disperse3D(
    sdss_dr7, '_disperse_03/bin/',
    H0, Om, Ol, Ok,
    clusters=sdss_opt_gr
)

In [10]:
DPS.count_cart_coords()

In [11]:
DPS.galaxies

,RA,DEC,Z,CX,CY,CZ
0,146.71421,-1.041304,0.021314,-78.414920,51.481140,-1.705001
1,147.17639,-0.354057,0.006436,-23.890216,15.410130,-0.175679
2,146.85676,-0.274040,0.022017,-81.132541,52.976931,-0.463453
3,146.61904,-0.434322,0.006087,-22.453840,14.794876,-0.203838
4,146.75170,-0.408392,0.035831,-131.450954,86.177445,-1.120371
...,...,...,...,...,...,...
61774,260.39196,31.679209,0.029946,-18.685193,-110.379299,69.085412
61775,260.64573,31.807949,0.033229,-20.147840,-122.308989,76.880623
61776,260.73569,31.341986,0.015230,-9.231003,-56.591395,34.920428
61777,261.10350,31.590881,0.025233,-14.618313,-93.387920,58.131507


In [12]:
SIGMA = 5
SMOOTH = 2
BOARD = 'smooth'
ANGLE = 30

In [13]:
DPS.apply_disperse(SIGMA, SMOOTH, BOARD, ANGLE)

>>> delaunay_3D starts
>>> mse starts
>>> skelconv starts
>>> read_skl_ascii_RaDecZ starts


In [14]:
input_, mask = DPS.get_seg_mask(3, 3)

100%|██████████| 61779/61779 [00:00<00:00, 123603.62it/s]


In [15]:
mask.sum()

5142.0

In [16]:
import UNet3D

In [17]:
train_dataloader = torch.utils.data.DataLoader(
    UNet3D.UNetDataset(input_, mask, 3, 12), 
    batch_size=8, 
    shuffle=True
)

In [18]:
lr = 0.001
model = UNet3D.Unet3DModel(in_channels=1, out_channels=1).double()
optim = torch.optim.Adam(params=model.parameters(), lr=lr)
criterion = UNet3D.DiceLoss()

In [19]:
trainer = UNet3D.Trainer(model, optim, criterion, 10)

In [ ]:
trainer.train(train_dataloader)